<a href="https://colab.research.google.com/github/JyzMinaBF/CriminalLaw/blob/main/Crimina_Law_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import pickle

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense, Embedding
from tensorflow.keras.utils import to_categorical
from urllib.request import urlretrieve

In [ ]:
urlretrieve("https://github.com/JyzMinaBF/CriminalLaw/raw/main/law1.pkl", "Criminal.pkl")

('Criminal.pkl', <http.client.HTTPMessage at 0x7f7739f5add0>)

In [ ]:
f = open('Criminal.pkl', 'rb')
CriminalLaw = pickle.load(f)
f.close()

In [ ]:
# text_lines = [x.lstrip('\u3000\u3000') for x in lines]

In [ ]:
# text = ''.join(text_lines)

In [ ]:
# text

### 訓練 tokenizer

In [ ]:
tokenizer = Tokenizer(char_level=True)

In [ ]:
tokenizer.fit_on_texts([CriminalLaw])

試用一下

In [ ]:
tokenizer.texts_to_sequences(["行為之處罰"])

[[20, 29, 2, 13, 17]]

變成文字。

In [ ]:
''.join(tokenizer.sequences_to_texts([[0]])[0].split())

NameError: ignored

計算最大 id, 即不同的字的數目。

In [ ]:
max_id = len(tokenizer.word_index)
print(max_id)

972


In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([CriminalLaw]))-1

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded)

In [ ]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:,1:]))

In [ ]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [ ]:
dataset = dataset.prefetch(1)

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=[None, max_id]))

In [ ]:
model.add(TimeDistributed(Dense(max_id, activation="softmax")))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')

In [ ]:
history = model.fit(dataset, epochs=10)

Epoch 1/10
912/912 [==============================] - 751s 821ms/step - loss: 2.7001
Epoch 2/10
912/912 [==============================] - 757s 828ms/step - loss: 2.1619
Epoch 3/10
912/912 [==============================] - 757s 828ms/step - loss: 1.7318
Epoch 4/10
912/912 [==============================] - 744s 814ms/step - loss: 1.3783
Epoch 5/10
912/912 [==============================] - 748s 818ms/step - loss: 1.1341
Epoch 6/10
912/912 [==============================] - 745s 815ms/step - loss: 0.9570
Epoch 7/10
912/912 [==============================] - 759s 830ms/step - loss: 0.8183
Epoch 8/10
912/912 [==============================] - 751s 821ms/step - loss: 0.7082
Epoch 9/10
912/912 [==============================] - 753s 824ms/step - loss: 0.6211
Epoch 10/10
912/912 [==============================] - 746s 815ms/step - loss: 0.5522


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 128)         563712    
                                                                 
 time_distributed (TimeDistr  (None, None, 972)        125388    
 ibuted)                                                         
                                                                 
Total params: 689,100
Trainable params: 689,100
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 把 model 存起來

In [ ]:
model.save('CriminalLaw_rnn_weights.h5')

In [ ]:
from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
#save the model architecture to JSON file
with open('CriminalLaw_rnn.json', 'w') as json_file:
    json_file.write(json_model)


In [ ]:
!mkdir -p saved_model
model.save('content/drive/My Drive/saved_model/CriminalLaw_rnn')

INFO:tensorflow:Assets written to: content/drive/My Drive/saved_model/CriminalLaw_rnn/assets


INFO:tensorflow:Assets written to: content/drive/My Drive/saved_model/CriminalLaw_rnn/assets


#### 把 tokenizer 存起來

In [ ]:
import pickle

In [ ]:
f = open('dream_tokenizer.pkl', 'wb')
pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)
f.close()